In [1]:
# import data and functions module
import tabulate
from functions import *
path_308 = 'data/308.15.txt'
path_358 = 'data/358.15.txt'
path_408 = 'data/408.15.txt'
path_313 = 'data/313.15.txt'

data_308 = pd.read_csv(path_308, delimiter='\t')
data_358 = pd.read_csv(path_358, delimiter='\t')
data_408 = pd.read_csv(path_408, delimiter='\t')
data_313 = pd.read_csv(path_313, delimiter='\t')

Q2. Table 1. molar volume, compressibility factor for CO 2 at 35o C, 85o C and 135o C over 0.18-18 MPa


In [2]:
# Define pressure and temperature values as NumPy arrays
P_vals = 1000000*np.array([0.18, 2, 4, 6, 8, 10, 12, 14, 16, 18]) # Pa
T_vals = np.array([308.15, 358.15, 408.15]) # K
datasets = [data_308, data_358, data_408]

table1 = pd.DataFrame(columns=["P (MPa)", "v(m3/mol)", "Z", "PR-Z", "% Diff Z"])
i = 0 # index for datasets
j = 0 # index for NIST data
k = 0 # index for table1

for T in T_vals:
  for P in P_vals:
    table1.at[k, "P (MPa)"] = P/1000000

    PRZ = calc_PRZ(T, P) # Calculate PR-Z using Peng-Robinson EOS
    table1.at[k, "PR-Z"] = PRZ

    data = datasets[i]
    Z = calc_Z(T, P, data["Volume (m3/mol)"][j]) # Calculate Z using NIST data
    table1.at[k, "v(m3/mol)"] = data["Volume (m3/mol)"][j]
    table1.at[k, "Z"] = Z

    percent_diffz = percent_diff(Z,PRZ) # Calculate percent difference between PR-Z and NIST Z
    table1.at[k, "% Diff Z"] = percent_diffz

    if j < 9:
      j += 1
    k += 1  
  j = 0
  i += 1
table1.to_markdown("table1.md", index=False)

Q2. Table 2. Enthalpy difference for CO2 at specified P and T

In [44]:
P2_vals = 1000000*np.array([2, 8, 16])
table2 = pd.DataFrame(columns=["P (MPa)", "Delta H T85-T35 (kJ/mol)", "Delta Hdep T85-T35 (kJ/mol)", "Delta H T135-T35 (kJ/mol)", 
                               "Delta Hdep T135-T35 (kJ/mol)"])

l=0 # index for table2
m = [1, 4, 8] # index in data, starting at P=2 MPa

for P in P2_vals: 
    table2.at[l, "P (MPa)"] = P/1000000
    DeltaH1 = data_358["Enthalpy (kJ/mol)"][m[l]] - data_308["Enthalpy (kJ/mol)"][m[l]]
    table2.at[l, "Delta H T85-T35 (kJ/mol)"] = DeltaH1

    DeltaHid1 = calc_idealH(308.15, 358.15)
    Hres308 = res_enth(308.15, P, calc_PRZ(308.15, P))
    Hres358 = res_enth(358.15, P, calc_PRZ(358.15, P))
    DeltaHdep1 = DeltaHid1 + (Hres358 - Hres308)
    table2.at[l, "Delta Hdep T85-T35 (kJ/mol)"] = DeltaHdep1

    DeltaH2 = data_408["Enthalpy (kJ/mol)"][m[l]] - data_308["Enthalpy (kJ/mol)"][m[l]]
    table2.at[l, "Delta H T135-T35 (kJ/mol)"] = DeltaH2

    DeltaHid2 = calc_idealH(308.15, 408.15)
    Hres408 = res_enth(408.15, P, calc_PRZ(408.15, P))
    DeltaHdep2 = DeltaHid2 + (Hres408 - Hres308)
    table2.at[l, "Delta Hdep T135-T35 (kJ/mol)"] = DeltaHdep2

    l += 1
table2.to_markdown("table2.md", index=False)

Q4 Finding the number of compression stages with intercooling

In [47]:
P = .18 # MPa, starting pressure
inlet_P= []
while P <= 18:
    inlet_P.append(round(P,3))
    P = P*3
print(len(inlet_P))

5


Q5. Calculations for H_in, H_out real

In [48]:
outlet_P = [0.54, 1.62, 4.86, 14.58, 18] # MPa
outlet_T = [408.91, 409.66, 411.92, 413.78, 318.13] # K, at real outlet enthalpy, manually found using NIST
H_in = [] # kJ/mol, corresponding to inlet_P at 40 oC
entropy = [] # J/mol-K, corresponding to inlet_P at 40 oC
H_s = [25.998, 25.809, 25.224, 23.116, 12.804] # kJ/mol, at outlet_P at entropy, manually found using NIST and added to isentropic.txt
H_out = [] # kJ/mol, actual enthalpy at outlet_P
for p in inlet_P:
    row = data_313[data_313["Pressure (MPa)"] == p] # find row in data_313 with matching pressure
    entropy.append(row["Entropy (J/mol*K)"].iloc[0]) # append entropy to list, constant for each stage
    H_in.append(row["Enthalpy (kJ/mol)"].iloc[0]) # append enthalpy to list 
for a in range(len(outlet_P)): 
    H_out.append(Hreal_out(H_in[a], H_s[a])) # calculate actual enthalpy at outlet_P

Q5. Table 3: Welec done at each stage of compression for CO2

In [50]:
table3 = pd.DataFrame(columns=["Inlet P (MPa)", "Outlet P (MPa)", "Inlet T (oC)", "Outlet T (oC)","Welec (MW)"])
for i in range(5):
    table3.at[i, "Inlet P (MPa)"] = inlet_P[i]
    table3.at[i, "Outlet P (MPa)"] = outlet_P[i]
    table3.at[i, "Inlet T (oC)"] = 40
    table3.at[i, "Outlet T (oC)"] = outlet_T[i] - 273.15
    table3.at[i, "Welec (MW)"] = calc_work(H_in[i], Hreal_out(H_in[i], H_s[i]), calc_mol_flow())
table3.to_markdown("table3.md", index=False)

Q6. Table 4: Heat duty of inter-stage coolers using dep function expression and ideal gas heat capacity (Cp)

In [51]:
t = 0 # index for inlet_T, which is equal to outlet_T
table4 = pd.DataFrame(columns=["Inlet P (MPa)", "Outlet P (MPa)", "Inlet T (oC)", "Outlet T (oC)", "Q (MW)"])
for P in outlet_P:
    table4.at[t, "Inlet P (MPa)"] = P
    table4.at[t, "Outlet P (MPa)"] = P
    table4.at[t, "Inlet T (oC)"] = outlet_T[t] - 273.15
    table4.at[t, "Outlet T (oC)"] = 40
    Q = calc_Q(outlet_T[t], 40+273.15, P*1000000, calc_mol_flow())
    table4.at[t, "Q (MW)"] = Q
    t += 1
table4.to_markdown("table4.md", index=False)

Q7. Calculations for operating expense for compression only

In [57]:
total_cost = 0 # $/ton, where cost is 16.147 cents/ kWh [2]
for i in range(5):
    cost=table3.at[i, "Welec (MW)"]*16.147*1000*24/(100*400)
    total_cost += cost
print(total_cost)

14.382163996528673


Q8. Percent of carbon credit attributed to CO2 compression

In [58]:
percent = total_cost/60 *100 # $60/ ton is the carbon credit
print(percent)

23.97027332754779
